In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
import numpy as np
from types import SimpleNamespace
import pickle
from concurrent.futures import ThreadPoolExecutor

from utils.bench import mk_fname, gen_stats_report, plot_results, process_bunch, slurp_lines

def without(xx, skip):
    return SimpleNamespace(**{k: v for k,v in xx.__dict__.items() if k not in skip})

In [ ]:
pp = SimpleNamespace(tile=(-9,-18),
                     block=(8,2), 
                     block_shape=(256,256),
                     dtype='uint8',
                     nthreads=1,
                     band=1
                    )

In [ ]:
files = slurp_lines('s3_tile_{:d}_{:d}_{}.txt', *pp.tile, 'zip')
files[:3], len(files), pp.block_shape

In [ ]:
pp.nthreads = 16
pool = ThreadPoolExecutor(max_workers=pp.nthreads)

In [ ]:
%%time
xx = process_bunch(files, pp, pool=pool)

In [ ]:
xx.t_total, xx.data.shape

In [ ]:
fnames = {ext:mk_fname(xx.params, ext=ext, prefix='M5XL_ZIP') for ext in ['pickle', 'npz']}

pickle.dump(without(xx, ['data']),
            open(fnames['pickle'], 'wb'))

np.savez(fnames['npz'], data=xx.data)

fnames

In [ ]:
print(gen_stats_report(xx))
plot_results(xx.stats, fig=plt.figure(figsize=(12,8)));